In [1]:
!pip install fastapi uvicorn pyngrok nest_asyncio torch transformers accelerate peft bitsandbytes

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from fastapi import FastAPI, Request
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import json

# ==========================================
# 2. 모델 로드 (LoRA 방식 그대로 복원)
# ==========================================
# ⚠️ 원래 학습할 때 썼던 베이스 모델 ID를 정확히 넣으세요
BASE_MODEL_ID = "Qwen/Qwen3-4B-Instruct-2507"
LORA_ID = "kwkun/my-rule-ai-lora"

print("📥 베이스 모델 로딩 중... (LoRA 방식)")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)

# 베이스 모델 로드 (GPU 직결)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.float16,
    device_map="cuda",
    trust_remote_code=True
)

print(f"🔗 LoRA 어댑터 연결 중... ({LORA_ID})")
model = PeftModel.from_pretrained(base_model, LORA_ID)
model.eval() # 추론 모드 확정

print("✅ 모델 로딩 완료! (LoRA 상태)")

📥 베이스 모델 로딩 중... (LoRA 방식)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

🔗 LoRA 어댑터 연결 중... (kwkun/my-rule-ai-lora)
✅ 모델 로딩 완료! (LoRA 상태)


In [ ]:
import threading
import uvicorn
from fastapi import FastAPI, Request
from pyngrok import ngrok
import nest_asyncio
import json

# ==========================================
# 3. FastAPI 서버 설정
# ==========================================
app = FastAPI()

SYSTEM_PROMPT = """
너는 한국 형사 명예훼손 판결 데이터를 학습하는 법률 특화 모델이다.
입력으로는 사건의 사실관계 요약(사실관계)만 주어진다.
너는 이 사실관계를 바탕으로 다음 항목을 JSON 형식으로 예측해야 한다.
반드시 아래 키만 포함된 하나의 JSON 객체만 출력하라.

{
  "판단": "유죄" | "무죄" | "일부유죄" | "기타",
  "형량": {
    "형종": "징역" | "벌금" | "무죄" | "기타",
    "형기_월": 정수 또는 null,
    "벌금액": 정수 또는 null,
    "집행유예_여부": true 또는 false 또는 null,
    "집행유예_기간_월": 정수 또는 null,
    "사회봉사_시간": 정수 또는 null,
    "추가_조건": 문자열 (없으면 빈 문자열 "")
  },
  "양형이유": "판결문 양형 이유를 요약한 한국어 문장 (없으면 빈 문자열)",
  "casenames": ["사건 유형(예: '명예훼손')"],
  "죄명": ["정보통신망이용촉진및정보보호등에관한법률위반(명예훼손)" 등]
}
"""

@app.post("/predict")
async def predict(request: Request):
    data = await request.json()
    input_text = data.get("inputs", "")

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"사실관계:\n{input_text}"}
    ]

    # LoRA 학습 때와 동일한 포맷팅 함수 사용
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,  # 결정론적 결과
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            temperature=0.1   # 필요시 조정
        )

    generated_ids = outputs[0][inputs.input_ids.shape[1]:]
    response_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

    # JSON 문자열만 깔끔하게 추출
    clean_text = response_text.replace("```json", "").replace("```", "").strip()

    return {"generated_text": clean_text}

# ==========================================
# 4. 서버 시작 (ngrok + Threading)
# ==========================================
# 사용자 제공 토큰 및 도메인 설정
NGROK_TOKEN = "자신의 ngrok 토큰 입력"
ngrok.set_auth_token(NGROK_TOKEN)

# 기존 터널 정리
ngrok.kill()

# 고정 도메인으로 터널 열기
public_url = ngrok.connect(8000, domain="자신의 ngrok URL 입력")

print(f"\n🚀 [데모용 API 주소]: {public_url.public_url}/predict")
print("웹 서비스 코드의 API_URL을 위 주소로 바꾸세요!\n")

# ✨ [핵심 수정] 별도 쓰레드에서 서버 실행 (Colab 충돌 방지)
def start_server():
    uvicorn.run(app, port=8000)

thread = threading.Thread(target=start_server)
thread.start()


🚀 [데모용 API 주소]: https://chasmed-sariah-rainily.ngrok-free.dev/predict
웹 서비스 코드의 API_URL을 위 주소로 바꾸세요!

